# None Seldon/Serve Example HuggingFace
This is the none Seldon (local implementation) of what I'm going to implement in the next section on top of Seldon core

In [1]:
from transformers import GPT2Tokenizer, TFGPT2Model
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2Model.from_pretrained('gpt2', from_pt=True)
model.save_pretrained("./tfgpt2model", saved_model=True)


text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)

count = 0
max_gen_len = 10
gen_sentence = text
while count < max_gen_len:
    input_ids = tokenizer.encode(gen_sentence, return_tensors="tf")
    shape = input_ids.shape.as_list()
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# input_text = "I enjoy working in Seldon"
# count = 0
# max_gen_len = 10
# gen_sentence = input_text

# TODO start with the source notebook
# TODO find out how to translate back logits into results https://huggingface.co/course/chapter6/3?fw=pt
# TODO read the theory of paper
# TODO read this for theory https://huggingface.co/blog/how-to-generate

output

/home/cc/miniconda3/envs/central/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-09 18:00:15.541393: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-09 18:00:15.541447: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (k8s-cluster): /proc/driver/nvidia/version does not exist
2022-05-09 18:00:15.542793: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-09 18:00:15.568166: 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: ./tfgpt2model/saved_model/1/assets


INFO:tensorflow:Assets written to: ./tfgpt2model/saved_model/1/assets


In [ ]:
import numpy as np

while count < max_gen_len:
    input_ids = tokenizer.encode(gen_sentence, return_tensors="tf")
    shape = input_ids.shape.as_list()
    payload = {
        "inputs": [
            {
                "name": "input_ids",
                "datatype": "INT32",
                "shape": shape,
                "data": input_ids.numpy().tolist(),
            },
            {
                "name": "attention_mask",
                "datatype": "INT32",
                "shape": shape,
                "data": np.ones(shape, dtype=np.int32).tolist(),
            },
        ]
    }

    ret = requests.post(
        "http://localhost:8004/seldon/seldon/gpt2/v2/models/gpt2/infer", json=payload
    )

    try:
        res = ret.json()
    except:
        continue

    # extract logits
    logits = np.array(res["outputs"][1]["data"])
    logits = logits.reshape(res["outputs"][1]["shape"])

    # take the best next token probability of the last token of input ( greedy approach)
    next_token = logits.argmax(axis=2)[0]
    next_token_str = tokenizer.decode(
        next_token[-1:], skip_special_tokens=True, clean_up_tokenization_spaces=True
    ).strip()
    gen_sentence += " " + next_token_str
    count += 1

print(f"Input: {input_text}\nOutput: {gen_sentence}")

# Server Examples

## Triton Server+TensorFlow+Seldon

## Triton Server+Torch+Seldon

## Triton Server

## TfServing+Seldon

## TfServing

## TorchServe

## Kserve

# Load Testing